In [1]:
import pickle as pkl
from icecream import ic
from datasets import load_dataset
import pickle as pkl
from llama import Llama, Dialog
from typing import List, Optional

/home/mz/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [52]:
with open("article.txt", "r") as f:
    article = f.read()

In [14]:
import json

with open("summaries.json", "r") as f:
    summaries = json.load(f)

summaries

{'original': " Sure! Here is a summary of the article in three numbered sentences:\n\n1. The Palestinian Authority officially became the 123rd member of the International Criminal Court (ICC) on Wednesday, giving the court jurisdiction over alleged crimes in Palestinian territories.\n2. The ICC opened a preliminary examination into the situation in Palestinian territories in January, and the formal accession of the Palestinian Authority as a member of the court paves the way for possible war crimes investigations against Israelis.\n3. The ICC's decision to allow the Palestinians to join the body despite opposition from Israel and the United States is seen as a step towards greater justice and accountability for alleged crimes committed in the region.",
 'paraphrased': " Sure! Here is a summary of the article in three numbered sentences:\n\n1. The Palestinian Authority officially became the 123rd member of the International Criminal Court (ICC) on Wednesday, giving the court jurisdictio

In [20]:
with open("summaries.json", "w") as f:
    json.dump(summaries, f)

In [6]:
def ratePromptBad(article, summary):
    prompt = """

    You will be given one summary written for a news article. Your task is to rate the summary based on the following criteria:

    Evaluation Criteria:
    1. Read the news article carefully and identify the main topic and key points.
    2. Read the summary and compare it to the news article. Check if the summary covers the main topic and key points of the news article, and if it resents them in a clear and logical order.
    3. Rate the summary with 5 percentages, where each one represents how likely the summary is going to get a score from 1 to 5. For example, if you think the summary is 2% likely to get a score of 5, 3% likely to get a score of 4, 5% likely to get a score of 3, 10% likely to get a score of 2, and 80% likely to get a score of 1, you should rate the summary as 2, 3, 5, 10, 80.
    4. Only report back with your final rating. For example: 2, 3, 5, 10, 80

    Here is the article: {article}

    Here is the summary: {summary}
    """.format(article=article, summary=summary)

    return prompt

In [7]:
def ratePromptGood(article, summary):
    prompt = """

    You will be given one summary written for a news article. Your task is to rate the summary based on the following criteria:

    Evaluation Criteria:
    1. Read the news article carefully and identify the main topic and key points.
    2. Read the summary and compare it to the news article. Check if the summary covers the main topic and key points of the news article, and if it resents them in a clear and logical order.
    3. Rate the summary with 5 percentages, where each one represents how likely the summary is going to get a score from 1 to 5. For example, if you think the summary is 80% likely to get a score of 5, 10% likely to get a score of 4, 5% likely to get a score of 3, 3% likely to get a score of 2, and 1% likely to get a score of 1, you should rate the summary as 80, 10, 5, 3, 2.
    4. Only report back with your final rating. For example: 80,10,5,3,2

    Here is the article: {article}

    Here is the summary: {summary}
    """.format(article=article, summary=summary)

    return prompt

In [8]:
def rankPrompt(article, summary1, summary2):
    prompt = """

    You will be given two summaries written for a news article. Your task is to rank the summaries based on the following criteria:

    Evaluation Criteria:
    1. Read the news article carefully and identify the main topic and key points.
    2. Read the summaries and compare them to the news article. Check if the summaries covers the main topic and key points of the news article, and if they resent them in a clear and logical order.
    3. Rank the summaries based on how likely you would recommend them to others. For example, if you think the second summary is more likely to be recommended than the first summary, you should rank them as: summary 2, summary 1.
    4. Only report back with your final ranking. For example: summary 2, summary 1

    Here is the article: {article}

    Here is the summary 1: {summary1}

    Here is the summary 2: {summary2}

""".format(article=article, summary1=summary1, summary2=summary2)

    return prompt

In [47]:
def rankPrompt2(article, summary1, summary2):
    prompt = """

    You will be given two summaries written for a news article. Your task is to rank the summaries based on the following criteria:
    
    Rank the summaries based on how they meet the criteria defined here. For example, as long as summary 2 better meets the criteira, rank: summary 2, summary 1.

    Output Format: 
    Only report back with your final ranking. For example: summary 2, summary 1

    Evaluation Criteria:
    1. Read the news article carefully and identify the main topic and key points.
    2. Read the summaries and compare them to the news article. Check if the summaries covers the main topic and key points of the news article, and if they resent them in a clear and logical order.
    3. If both summaries meet the requirements above, then rank them based on how concise and clear they are. For example, both summary 1 and summary 2 cover the key points of a news article, but summary 2 is shorter in lenght and summary 2 uses more common language, you should recommend summary 2 over summary 1.
    4. If both summaries meet the requirements above, then rank them based on whether they are free from grammatical errors and typos. For example, if the first summary has more grammatical errors and typos than summary 2, then you should recommend summary 2 over summary 1.
    5. If both summaries meet the requirements above, then rank them based on how likely you would recommend them to others. For example, if you think the summary 2 is more likely to be recommended than the first summary, you should rank them as: summary 2, summary 1.

    Here is the article: {article}

    Here is the summary 1: {summary1}

    Here is the summary 2: {summary2}

""".format(article=article, summary1=summary1, summary2=summary2)

    return prompt

In [23]:
print(rankPrompt(article, summaries['para_para_did'], summaries['twistSummary']))



    You will be given two summaries written for a news article. Your task is to rank the summaries based on the following criteria:

    Evaluation Criteria:
    1. Read the news article carefully and identify the main topic and key points.
    2. Read the summaries and compare them to the news article. Check if the summaries covers the main topic and key points of the news article, and if they resent them in a clear and logical order.
    3. Rank the summaries based on how likely you would recommend them to others. For example, if you think the second summary is more likely to be recommended than the first summary, you should rank them as: summary 2, summary 1.
    4. Only report back with your final ranking. For example: summary 2, summary 1

    Here is the article: {'article': '(CNN)The Palestinian Authority officially became the 123rd member of the International Criminal Court on Wednesday, a step that gives the court jurisdiction over alleged crimes in Palestinian territories. 

In [48]:
print(rankPrompt2(article, summaries['para_para'], summaries['reverseSummary']))



    You will be given two summaries written for a news article. Your task is to rank the summaries based on the following criteria:
    
    Rank the summaries based on how they meet the criteria defined here. For example, as long as summary 2 better meets the criteira, rank: summary 2, summary 1.

    Output Format: 
    Only report back with your final ranking. For example: summary 2, summary 1

    Evaluation Criteria:
    1. Read the news article carefully and identify the main topic and key points.
    2. Read the summaries and compare them to the news article. Check if the summaries covers the main topic and key points of the news article, and if they resent them in a clear and logical order.
    3. If both summaries meet the requirements above, then rank them based on how concise and clear they are. For example, both summary 1 and summary 2 cover the key points of a news article, but summary 2 is shorter in lenght and summary 2 uses more common language, you should recommend s

In [53]:
from icecream import ic
print(ratePromptGood(article, original))



    You will be given one summary written for a news article. Your task is to rate the summary based on the following criteria:

    Evaluation Criteria:
    1. Read the news article carefully and identify the main topic and key points.
    2. Read the summary and compare it to the news article. Check if the summary covers the main topic and key points of the news article, and if it resents them in a clear and logical order.
    3. Rate the summary with 5 percentages, where each one represents how likely the summary is going to get a score from 1 to 5. For example, if you think the summary is 80% likely to get a score of 5, 10% likely to get a score of 4, 5% likely to get a score of 3, 3% likely to get a score of 2, and 1% likely to get a score of 1, you should rate the summary as 80, 10, 5, 3, 2.
    4. Only report back with your final rating. For example: 80,10,5,3,2

    Here is the article: (CNN)The Palestinian Authority officially became the 123rd member of the International Cri

In [14]:
def build(ckpt_dir,tokenizer_path,max_seq_len,max_batch_size):
    generator = Llama.build(
        ckpt_dir=ckpt_dir,
        tokenizer_path=tokenizer_path,
        max_seq_len=max_seq_len,
        max_batch_size=max_batch_size,
    )
    return generator

def call(generator, article, summary, max_gen_len,temperature,top_p):
    dialogs: List[Dialog] = []

    user_input = prompt(article, summary)

    dialogs.append([{"role": "user", "content": "{}".format(user_input)}])
        
    print(len(dialogs))

    
    results = generator.chat_completion(
        dialogs,  # type: ignore
        max_gen_len=max_gen_len,
        temperature=temperature,
        top_p=top_p,
    )

    #print(results)
    for result in results:
        # for msg in dialog:
            # print(f"{msg['role'].capitalize()}: {msg['content']}\n")
        # if name=='cnn_dailymail':
        #     final=prompt_output_parse_CNN(result['generation']['content'])

        # else:
        #     final=prompt_output_parse(result['generation']['content'])
        
        final=result['generation']['content']
    
    #print(final)
    return final
        

In [15]:

def main(
    ckpt_dir: str,
    tokenizer_path: str,
    temperature: float = 0.7,
    top_p: float = 0.9,
    max_seq_len: int = 8000,
    max_batch_size: int = 2,
    max_gen_len: Optional[int] = None,
):
    generator = build(ckpt_dir,tokenizer_path,max_seq_len,max_batch_size)

    dataset = load_dataset("xsum")
    article_key = 'document'
    summary_key = 'summary'
    data=dataset['test']

    with open("data_original/cnn/0.pkl", 'rb') as f:
        summary = pkl.load(f)
    
    with open("paraphrased_articles/cnn/0.pkl", 'rb') as f:
        article = pkl.load(f)

    out = call(generator, article, summary, max_gen_len,temperature,top_p)

    _ = ic(out)



In [20]:
import argparse
parser = argparse.ArgumentParser()
parser.add_argument("--local-rank", type=int)
args = parser.parse_args()

local_rank = args.local_rank

main('/home/mz/AIModels/llama/llama-2-7b-chat/', '/home/mz/Documents/github/Relevance-Paraphrasing-main/llama2/tokenizer.model')

usage: ipykernel_launcher.py [-h] [--local-rank LOCAL_RANK]
ipykernel_launcher.py: error: unrecognized arguments: --f=/home/mz/.local/share/jupyter/runtime/kernel-v2-4758692bF7uh8GNRCX.json


SystemExit: 2

/home/mz/.local/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3561: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
